# Limpieza de la Base de Datos

Este notebook contiene todas las funciones necesarias para limpiar y transformar la base en la versión manejable para el proyecto.

Los objetivos son los siguientes:
    
* Depurar los campos que contengan texto con algún error de escritura o caracteres inadecuados para el procesamiento.
* Fijar un tipo de dato adecuado para el manejo con otros archivos
* Compactar u organizar las columnas con reportes mensuales en un formato que permita una mejor visualización y organización de la información. Además de la reducion de su tamaño.


In [1]:
#Bibliotecas requeridas

from IPython.core.interactiveshell import InteractiveShell
from typing import Dict
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import gc
import warnings

#Se ignoran mensajes de cambios o warnings
warnings.filterwarnings("ignore")

#Configuración del Notebook
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 250)
pd.set_option('display.width', 50)

In [2]:
# CONSTANTES PARA EL PROCESO DE LIMPIEZA
SCHEMA={'CICLO':'category',
 'ID_RAMO':'category',
 'DESC_RAMO':'category',
 'ID_UR':'category',
 'DESC_UR':'category',
 'ID_ENTIDAD_FEDERATIVA':'category',
 'ENTIDAD_FEDERATIVA':'category',
 'ID_MUNICIPIO':'category',
 'MUNICIPIO':'category',
 'GPO_FUNCIONAL':'category',
 'DESC_GPO_FUNCIONAL':'category',
 'ID_FUNCION':'category',
 'DESC_FUNCION':'category',
 'ID_SUBFUNCION':'category',
 'DESC_SUBFUNCION':'category',
 'ID_AI':'category',
 'DESC_AI':'category',
 'ID_MODALIDAD':'category',
 'DESC_MODALIDAD':'category',
 'ID_PP':'category',
 'DESC_PP':'category',
 'MODALIDAD_PP':'category',
 'ID_PND':'category',
 'DESC_PND':'category',
 'OBJETIVO_PND':'category',
 'PROGRAMA_PND':'category',
 'DESC_PROGRAMA_PND':'category',
 'OBJETIVO_PROGRAMA_PND':'category',
 'DESC_OBJETIVO_PROGRAMA_PND':'category',
 'OBJETIVO_ESTRATEGICO':'category',
 'ID_NIVEL':'category',
 'DESC_NIVEL':'category',
 'INDICADOR_PND':'category',
 'TIPO_RELATIVO':'category',
 'FRECUENCIA':'category',
 'TIPO_INDICADOR':'category',
 'DIMENSION':'category',
 'UNIDAD_MEDIDA':'category',
 'SENTIDO':'category'
 }

DTIPO_INDICADOR={'Estratégico':'Estratégico',
 'Gestión':'Gestión',
 'Sesiones de Comité Técnico':'Sesiones de Comité Técnico',
 'Gestion':'Gestión',
 'SOLICITUDES DE SERVICIO':'Solicitudes de Servicio',
 'ECONOMIA':'Economía',
 'Estrategico':'Estratégico',
 'gestión':'Gestión',
 'Absoluto':'Absoluto',
 'Sectorial':'Sectorial',
 'Desempeño Operativo':'Desempeño Operativo',
 'GESTION':'Gestión',
 'ESTRATÉGICO':'Estratégico',
 'De Gestión':'Gestión',
 'Estratgico':'Estratégico'}

DDIMENSION={'Eficacia':'Eficacia',
 'Eficiencia':'Eficacia',
 'Economía':'Economía',
 'Calidad':'Calidad',
 'eficacia':'Eficacia',
 'ECONOMIA':'Economía',
 '0':'Sin Dato',
 'Servicios Personales':'Servicios Personales',
 'Económica':'Economía',
 'Eificacia':'Eficacia',
 'EFICACIA':'Eficacia',
 'Eficiciencia':'Eficiencia',

 'Es la suma ponderada de la proporción de las observaciones de alto impacto respecto del total de observaciones determinadas en las auditorías directas de alto impacto realizadas por el área de Auditoría Interna del OIC; la calidad de dichas observaciones, y la calidad de las recomendaciones que de éstas se derivan. (Eficacia)':'Eficacia',

 'Es un promedio ponderado que evalúa al OIC en la atención de quejas y denuncias. (Eficacia)':'Eficacia',

 'Mide las acciones de las Áreas de Responsabilidades en algunas de sus funciones primordiales: 1) el tiempo en la atención de los expedientes, 2) la resolución de expedientes y 3) la firmeza de las sanciones impuestas. (Eficacia)':'Eficacia',

 'PORCENTAJE DE SOLICITUDES DE PRÉSTAMO AUTORIZADAS':'Porcentaje de Solicitudes de Préstamo Autorizadas',

 'El Indicador de Mejora de la Gestión (IMG) evalúa las acciones realizadas por los OIC en sus instituciones de adscripción y en aquellas bajo su atención, así como los resultados alcanzados en las mismas. Específicamente, el indicador se orienta a evaluar la manera en que los OIC:\r\n\r\n- Promueven acciones orientadas al logro de resultados respecto a las vertientes comprometidas en sus Programas Anuales de Trabajo (PAT)2015, en materia de auditoría para el desarrollo y mejora de la gestión pública.':'Sin Datos'}



In [3]:
#VARIABLES DE CONFIGURACIÓN  Y FUNCIONES AUXILIARES

#Dirección del archivo
PATHFILE='/home/dlegorreta/Downloads/prog_avance_de_indicadores.csv' #Escribir la dirección o ruta del archivo


#Funciones Auxiliares

def cln_txt(str_inp:str)->str:

    str_inp=str_inp.replace(u'\xa0',u' ')
    str_inp=str_inp.replace(u'\n',u' ')
    str_inp=str_inp.replace(u'\r',u' ')
    txt=''.join([s for s in str_inp if not s in '!"#$%&\'()*+-;<=>?@[\\]^_`{|}~' ])
    return txt.replace('  ','').strip()



In [4]:
data=pd.read_csv(PATHFILE,encoding='latin1',low_memory=False,dtype=SCHEMA)

print("Información General del Archivo:\n")
print(data.info())


Información General del Archivo:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103674 entries, 0 to 103673
Columns: 245 entries, CICLO to LIGA
dtypes: category(39), float64(143), int64(2), object(61)
memory usage: 168.1+ MB
None


In [5]:
#Eliminacion de NIVEL== FID
data=data[data.DESC_NIVEL!='FID'].copy()

In [6]:
#LIMPIEZA DE LAS DESCRIPCIONES

data.DESC_RAMO=data.DESC_RAMO.apply(lambda x: cln_txt(str(x)))
data.DESC_UR=data.DESC_UR.apply(lambda x: cln_txt(str(x)))
data.DESC_AI=data.DESC_AI.apply(lambda x: cln_txt(str(x)))
data.DESC_PP=data.DESC_PP.apply(lambda x: cln_txt(str(x)))
data.OBJETIVO_PND=data.OBJETIVO_PND.apply(lambda x: cln_txt(str(x)))
data.DESC_OBJETIVO_PROGRAMA_PND=data.DESC_OBJETIVO_PROGRAMA_PND.apply(lambda x: cln_txt(str(x)))
data.OBJETIVO_ESTRATEGICO=data.OBJETIVO_ESTRATEGICO.apply(lambda x: cln_txt(str(x)))
data.DESC_MATRIZ=data.DESC_MATRIZ.apply(lambda x: cln_txt(str(x)))
data.DESC_OBJETIVO=data.DESC_OBJETIVO.apply(lambda x: cln_txt(str(x)))

In [7]:
#CAMBIO EN INDICADORES CON NOMBRES ERRONEOS
data.TIPO_INDICADOR=data.TIPO_INDICADOR.map(DTIPO_INDICADOR)
data.DIMENSION=data.DIMENSION.map(DDIMENSION)

In [8]:
#AJUSTE EN EL TIPO DE DATOS

data.ID_OBJETIVO=data.ID_OBJETIVO.astype('int')
data.ID_OBJETIVO_PADRE=data.ID_OBJETIVO_PADRE.fillna(-1).astype('int')
data.ID_INDICADOR_CICLO_ANTERIOR=data.ID_INDICADOR_CICLO_ANTERIOR.fillna(-1).astype('int')
data.CICLO_LINEA_BASE=data.CICLO_LINEA_BASE.fillna(-1).astype('int')


In [9]:
#LISTA DE COLUMNAS QUE SE AGRUPAN

META_MES_COL=data.columns[data.columns.str.startswith('META_MES')].tolist()
META_AJUSTADA_MES_COL=data.columns[data.columns.str.startswith('META_AJUSTADA_MES')].tolist()
AVANCE_MES_COL=data.columns[data.columns.str.startswith('AVANCE_MES')].tolist()
JUSTIFICACION_AJUSTE_MES_COL=data.columns[data.columns.str.startswith('JUSTIFICACION_AJUSTE_MES')].tolist()
AVANCE_CAUSA_MES_COL=data.columns[data.columns.str.startswith('AVANCE_CAUSA_MES')].tolist()
AVANCE_EFECTO_MES_COL=data.columns[data.columns.str.startswith('AVANCE_EFECTO_MES')].tolist()
AVANCE_OTROS_MOTIVOS_MES_COL=data.columns[data.columns.str.startswith('AVANCE_OTROS_MOTIVOS_MES')].tolist()

In [10]:
#META por mes
for i in range(12):
    data[f'RECORDS_META_MES{i+1}']=(data[f'META_MES{i+1}'].astype('string')+':'\
        +data[f'META_MES{i+1}_NUM'].astype('string')+':'+data[f'META_MES{i+1}_DEN']\
            .astype('string'))


In [11]:
#META AJUSTADA por mes
for i in range(12):
    data[f'RECORDS_META_AJUSTADA_MES{i+1}']=(data[f'META_MES{i+1}'].astype('string')\
        +':'+data[f'META_MES{i+1}_NUM'].astype('string')+':'+data[f'META_MES{i+1}_DEN']\
            .astype('string'))


In [12]:
#AVANCE por mes
for i in range(12):
    data[f'RECORDS_AVANCE_MES{i+1}']=(data[f'META_MES{i+1}'].astype('string')+':'+\
        data[f'META_MES{i+1}_NUM'].astype('string')+':'+data[f'META_MES{i+1}_DEN']\
            .astype('string'))



In [13]:
#JUSTIFICACION por mes

func='|'.join

data['JUSTIFICACIONES_AJUSTE_POR_MES']=data[JUSTIFICACION_AJUSTE_MES_COL]\
    .fillna('#').astype('str').apply(lambda x:func(x),axis=1)


In [14]:
#AVANCE CAUSA por mes

data['AVANCE_CAUSA_POR_MES']=data[AVANCE_CAUSA_MES_COL].fillna('#').astype('str')\
.apply(lambda x:func(x),axis=1)


In [15]:
#AVANCE EFECTO por mes
data['AVANCE_EFECTO_POR_MES']=data[AVANCE_EFECTO_MES_COL].fillna('#').astype('str')\
.apply(lambda x:func(x),axis=1)


In [16]:
#AVANCE OTROS MOTIVOS por mes
data['AVANCE_OTROS_MOTIVOS_POR_MES']=data[AVANCE_OTROS_MOTIVOS_MES_COL].fillna('#')\
.astype('str').apply(lambda x:func(x),axis=1)


In [17]:
#Eliminación de Columnas
data.drop(labels=META_MES_COL+META_AJUSTADA_MES_COL+AVANCE_MES_COL,inplace=True,axis=1)
data.drop(labels=JUSTIFICACION_AJUSTE_MES_COL+AVANCE_CAUSA_MES_COL+\
    AVANCE_EFECTO_MES_COL+AVANCE_OTROS_MOTIVOS_MES_COL,inplace=True,axis=1)
#Se visualizan aspecto generales del archivo
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 98701 entries, 0 to 103670
Columns: 129 entries, CICLO to AVANCE_OTROS_MOTIVOS_POR_MES
dtypes: category(33), float64(29), int64(6), object(25), string(36)
memory usage: 76.9+ MB


In [18]:
#Se gurda el archivo
path1=Path('.').resolve().parent/'data'/'base'
data.reset_index().to_feather(path1.as_posix())#Para version feather


Ok!!!
